In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/home/xs/tensorflow_test/sycu-tensorflow-train/research/object_detection")
from object_detection.utils import ops as utils_opsjiqi

from utils import label_map_util
from utils import visualization_utils as vis_util

In [2]:
model_file_path = "./ssd_mobilenet_v1_sycu_20210328/frozen_inference_graph.pb"
label_map_path = "./ssd_mobilenet_v1_sycu_20210328/sycu_label_map.pbtxt"
NUM_CLASSES = 4
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [3]:
import cv2
import time
frame = cv2.imread('./ball.png')

In [4]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        #graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
 
    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="")
    return graph


In [5]:
detect_graph = load_graph(model_file_path)
print(detect_graph)

In [6]:
detect_graph = load_graph(model_file_path)

with tf.Session(graph=detect_graph) as sess:
    image_np_expanded = np.expand_dims(frame, axis=0)
    image_tensor = detect_graph.get_tensor_by_name('image_tensor:0')

    boxes = detect_graph.get_tensor_by_name('detection_boxes:0')
    scores = detect_graph.get_tensor_by_name('detection_scores:0')
    classes = detect_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detect_graph.get_tensor_by_name('num_detections:0')
    
    (num_detections,boxes, scores, classes) = sess.run(
                [num_detections, boxes, scores, classes],
                feed_dict={image_tensor: image_np_expanded})
    
    vis_util.visualize_boxes_and_labels_on_image_array(
                frame, np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores), category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
 
    cv2.imshow('object detection', frame)
    k = cv2.waitKey(0)
    if k == ord('q'):
        cv2.destroyAllWindows()